# SPIdepth training
This notebook tries to train the SPIdepth network.

## Creating small split files
Since we are going to make a simple test, is not necessary to use all the Kitti dataset.So, let's create new split files with a small set of train and validation samples.

In [2]:
import os
import random

def sample_and_write_file(input_path, output_path, sample_ratio=0.1, seed=42):
    """
    Reads a list of file paths, selects a subset while maintaining proportions, and writes to a new file.
    
    Args:
        input_path (str): Path to the original split file (train_files.txt or val_files.txt).
        output_path (str): Path to save the reduced split file.
        sample_ratio (float): Fraction of original samples to keep (e.g., 0.1 for 10%).
        seed (int): Random seed for reproducibility.
    """
    # Read the original file lines
    with open(input_path, "r") as f:
        lines = f.readlines()
    
    # Set random seed for reproducibility
    random.seed(seed)

    # Compute number of samples to keep
    num_samples = max(1, int(len(lines) * sample_ratio))  # Ensure at least 1 sample
    
    # Randomly select a subset of lines
    sampled_lines = random.sample(lines, num_samples)

    # Write the sampled lines to the new file
    with open(output_path, "w") as f:
        f.writelines(sampled_lines)
    
    print(f"Created {output_path} with {num_samples} samples.")

In [4]:
# Paths to original files
split_dir = "splits/eigen_zhou"  
train_file = os.path.join(split_dir, "train_files.txt")
val_file = os.path.join(split_dir, "val_files.txt")

# Paths to new small dataset files
train_file_small = os.path.join(split_dir, "train_files_small.txt")
val_file_small = os.path.join(split_dir, "val_files_small.txt")

# Adjust the sample ratio (e.g., 0.1% of the original dataset)
sample_ratio = 0.001

In [6]:
# Generate the small dataset files
sample_and_write_file(train_file, train_file_small, sample_ratio)
sample_and_write_file(val_file, val_file_small, sample_ratio)

Created splits/eigen_zhou\train_files_small.txt with 71 samples.
Created splits/eigen_zhou\val_files_small.txt with 4 samples.
